In [1]:
import os,sys
from sklearn import preprocessing

import json
import codecs
import yaml
import numpy as np
import pandas as pd

sys.path.append('../utils')
from cudaMetrics import *

In [2]:
#
# read metrics folder
#
metrics_folder='home-gpu0'
appTrace = os.listdir(metrics_folder)
print len(appTrace)

76


In [3]:
appTrace

['metrics_cudasdk_interval.csv',
 'metrics_rodinia_hotspot.csv',
 'metrics_rodinia_heartwall.csv',
 'metrics_cudasdk_dxtc.csv',
 'metrics_cudasdk_MCSingleAsianOptionP.csv',
 'metrics_parboil_stencil.csv',
 'metrics_cudasdk_mergeSort.csv',
 'metrics_rodinia_hybridsort.csv',
 'metrics_shoc_lev1BFS.csv',
 'metrics_parboil_mriq.csv',
 'metrics_rodinia_b+tree.csv',
 'metrics_poly_covariance.csv',
 'metrics_cudasdk_shflscan.csv',
 'metrics_cudasdk_transpose.csv',
 'metrics_poly_gemm.csv',
 'metrics_cudasdk_dwtHaar1D.csv',
 'metrics_cudasdk_MCEstimatePiQ.csv',
 'metrics_poly_2dconv.csv',
 'metrics_cudasdk_convolutionFFT2D.csv',
 'metrics_rodinia_dwt2d.csv',
 'metrics_parboil_lbm.csv',
 'metrics_cudasdk_convolutionSeparable.csv',
 'metrics_cudasdk_concurrentKernels.csv',
 'metrics_poly_fdtd2d.csv',
 'metrics_cudasdk_sortingNetworks.csv',
 'metrics_poly_gesummv.csv',
 'metrics_rodinia_lud.csv',
 'metrics_poly_correlation.csv',
 'metrics_parboil_bfs.csv',
 'metrics_cudasdk_MCEstimatePiInlineP.cs

In [4]:
app_kernel_metrics_dd = {}
for i,f in enumerate(appTrace):
    csvfile  = metrics_folder + '/' + f
    #print f
    sep_str = f.split('_')
    appName = sep_str[-1][:-4]
    
    df_app = read_trace(csvfile)
    
    kernelList = df_app['Kernel'].unique()
    
    #
    # save metrics set
    #
    if i == 0:
        metricsAll = {}
        metricsAll['current'] = list(df_app['Metric Name'].unique())
        with codecs.open('current_metrics.json', 'w', encoding='utf-8') as outfile:
            json.dump(metricsAll, outfile, ensure_ascii=False)
    
    # extract metrics for each kernel
    kernel_metrics_dd = {}
    for kernName in kernelList:
        df_kern = df_app.loc[df_app['Kernel'] == kernName]
        # go through each metrics and read the avg value
        metrics_dd = {}
        for index, rows in df_kern.iterrows():
            local_metric_name = rows['Metric Name']
            local_metric_value = rows['Avg']
            try:
                adjustedV = adjust_metric(local_metric_name, local_metric_value)
            except:
                print("Unexpected error!\nfile: {}\nmetric: {}\nvalue: {}".format(currentFile, 
                    local_metric_name, local_metric_value))
                sys.exit(0)
            
            metrics_dd[local_metric_name] = adjustedV
        # update 
        kernel_metrics_dd[kernName] = metrics_dd
    # update
    app_kernel_metrics_dd[appName] = kernel_metrics_dd
    #break

In [5]:
print("applications : {}".format(len(app_kernel_metrics_dd)))

totalKernNum = 0
for app, kern_metrics_dd in app_kernel_metrics_dd.iteritems():
    #print len(kern_metrics_dd)
    totalKernNum += len(kern_metrics_dd)

print("kernels : {}".format(str(totalKernNum)))

applications : 76
kernels : 238


### example

In [6]:
app_kernel_metrics_dd['scan']

{'scanExclusiveShared(uint4*, uint4*, unsigned int)': {'achieved_occupancy': 0.945988,
  'atomic_transactions': 0.0,
  'atomic_transactions_per_request': 0.0,
  'branch_efficiency': 1.0,
  'cf_executed': 419719.0,
  'cf_fu_utilization': 0.1,
  'cf_issued': 419719.0,
  'double_precision_fu_utilization': 0.0,
  'dram_read_throughput': 43.566339,
  'dram_read_transactions': 851990.0,
  'dram_utilization': 0.9,
  'dram_write_throughput': 43.50394,
  'dram_write_transactions': 850785.0,
  'ecc_throughput': 0.0,
  'ecc_transactions': 0.0,
  'eligible_warps_per_cycle': 1.124316,
  'flop_count_dp': 0.0,
  'flop_count_dp_add': 0.0,
  'flop_count_dp_fma': 0.0,
  'flop_count_dp_mul': 0.0,
  'flop_count_sp': 0.0,
  'flop_count_sp_add': 0.0,
  'flop_count_sp_fma': 0.0,
  'flop_count_sp_mul': 0.0,
  'flop_count_sp_special': 0.0,
  'flop_dp_efficiency': 0.0,
  'flop_sp_efficiency': 0.0,
  'gld_efficiency': 1.0,
  'gld_requested_throughput': 43.564476,
  'gld_throughput': 43.564476,
  'gld_transaction

### offline training

In [7]:
#
# read metrics
#
with open('current_metrics.json', 'r') as metricsFile:
     metricsAll = yaml.safe_load(metricsFile)
        
TargetMetrics = metricsAll['current']

In [8]:
featMatCols = ['KernelName']         
featMatCols.extend(TargetMetrics)

In [9]:
featMatCols

['KernelName',
 'sm_efficiency',
 'achieved_occupancy',
 'ipc',
 'issued_ipc',
 'inst_per_warp',
 'branch_efficiency',
 'warp_execution_efficiency',
 'warp_nonpred_execution_efficiency',
 'inst_replay_overhead',
 'issue_slot_utilization',
 'shared_load_transactions_per_request',
 'shared_store_transactions_per_request',
 'local_load_transactions_per_request',
 'local_store_transactions_per_request',
 'gld_transactions_per_request',
 'gst_transactions_per_request',
 'shared_store_transactions',
 'shared_load_transactions',
 'local_load_transactions',
 'local_store_transactions',
 'gld_transactions',
 'gst_transactions',
 'dram_read_transactions',
 'dram_write_transactions',
 'global_hit_rate',
 'local_hit_rate',
 'gld_requested_throughput',
 'gst_requested_throughput',
 'gld_throughput',
 'gst_throughput',
 'dram_read_throughput',
 'dram_write_throughput',
 'tex_cache_throughput',
 'local_load_throughput',
 'local_store_throughput',
 'shared_load_throughput',
 'shared_store_throughput',

In [10]:
#
#  totalKernNum  x featureDim
#
df_featureMat = pd.DataFrame(index=np.arange(0, totalKernNum), columns=featMatCols)
print df_featureMat.shape

(238, 114)


In [11]:
#
# export data to data frame, so that we can export to csv file easily
#
rowId = 0
for appName, kern_metrics_dd in app_kernel_metrics_dd.iteritems():
    for kernName, metrics_dd in kern_metrics_dd.iteritems():
        app_kern_name = '[[ '+ str(appName) + ' ]] ' + str(kernName)
        
        # add kernel name to the list
        df_featureMat.loc[rowId, 'KernelName'] = app_kern_name

        # add other metrics according to the column order
        for eachMetric in TargetMetrics:
            try:                
                df_featureMat.loc[rowId, eachMetric] = metrics_dd[eachMetric]
            except Exception as e:
                print e.message, e.args
                print('ERROR!! App = {}, Kernel={}'.format(appName, kernName))
                sys.exit(0)

        rowId += 1

In [12]:
df_featureMat

,KernelName,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,[[ simpleCUFFTcallback ]] void spRadix0008A::k...,0.114939,0.041359,0.242014,0.065119,55.875,1,0.41443,0.413031,0.067114,...,0.1,0,0,0.1,0,0,0.1,0.063891,0.00014297,0
1,[[ simpleCUFFTcallback ]] void composite_2way_...,0.0860442,0.017421,0.048317,0.119726,226.5,1,0.264694,0.258485,0.024283,...,0.1,0,0,0.1,0,0,0.1,0.110953,8.755e-05,0
2,[[ simpleCUFFTcallback ]] void spRadix0007A::k...,0.123655,0.025864,0.071873,0.051377,95.375,1,0.360092,0.357798,0.073394,...,0.1,0,0,0.1,0,0,0.1,0.04962,0.00010094,0
3,[[ boxFilterNPP ]] void ForEachTupleByteQuad<u...,0.770417,0.354202,1.81824,1.91955,1878.21,0.814055,0.586253,0.581444,0.001604,...,0.3,0,0,0.1,0,0,0.1,3.8497,0.00293605,0
4,"[[ fdtd2d ]] fdtd_step1_kernel(float*, float*,...",0.995225,0.882651,0.983039,0.983087,34.9966,1,1,0.942866,0.000226,...,0.4,0,0,0.1,0,0,0.1,1.45499,0.0104764,0
5,"[[ fdtd2d ]] fdtd_step3_kernel(float*, float*,...",0.996192,0.871596,0.933717,0.933502,44,1,1,0.977273,0.00021,...,0.5,0,0,0.1,0,0,0.1,1.48581,0.0132121,0
6,"[[ fdtd2d ]] fdtd_step2_kernel(float*, float*,...",0.995252,0.885458,1.06898,1.06905,38,1,0.999704,0.973402,0.000214,...,0.4,0,0,0.1,0,0,0.1,1.75411,0.0104921,0
7,"[[ scan ]] scanExclusiveShared(uint4*, uint4*,...",0.995315,0.945988,1.03611,1.03628,98.6471,1,1,0.979783,0.000257,...,0.3,0,0,0.1,0,0,0.1,1.12432,0,0
8,"[[ scan ]] uniformUpdate(uint4*, unsigned int*)",0.995115,0.904988,0.216469,0.216553,20.625,1,0.970644,0.970455,0.000393,...,0.3,0,0,0.1,0,0,0.1,0.219366,0,0
9,"[[ scan ]] scanExclusiveShared2(unsigned int*,...",0.773095,0.495313,0.196354,0.203544,87.5,1,1,0.962367,0.037633,...,0.1,0,0,0.1,0,0,0.1,0.211818,0,0


In [16]:
#
# drop the 1st column 
#

df_featureMat = df_featureMat.convert_objects(convert_numeric=True)

df_featureMat_cp = df_featureMat.copy()
df_featureMat_cp = df_featureMat_cp.drop('KernelName', axis=1)   # remove the 1st column

/home/leiming/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """


In [17]:
df_featureMat_cp`b

,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,issue_slot_utilization,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,0.114939,0.041359,0.242014,0.065119,5.587500e+01,1.000000,0.414430,0.413031,0.067114,0.015768,...,0.1,0.000000,0.0,0.1,0.0,0.0,0.1,0.063891,1.429700e-04,0.000000
1,0.086044,0.017421,0.048317,0.119726,2.265000e+02,1.000000,0.264694,0.258485,0.024283,0.027674,...,0.1,0.000000,0.0,0.1,0.0,0.0,0.1,0.110953,8.755000e-05,0.000000
2,0.123655,0.025864,0.071873,0.051377,9.537500e+01,1.000000,0.360092,0.357798,0.073394,0.012107,...,0.1,0.000000,0.0,0.1,0.0,0.0,0.1,0.049620,1.009400e-04,0.000000
3,0.770417,0.354202,1.818238,1.919550,1.878211e+03,0.814055,0.586253,0.581444,0.001604,0.429939,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,3.849698,2.936050e-03,0.000000
4,0.995225,0.882651,0.983039,0.983087,3.499658e+01,1.000000,1.000000,0.942866,0.000226,0.231736,...,0.4,0.000000,0.0,0.1,0.0,0.0,0.1,1.454992,1.047638e-02,0.000000
5,0.996192,0.871596,0.933717,0.933502,4.400000e+01,1.000000,1.000000,0.977273,0.000210,0.206861,...,0.5,0.000000,0.0,0.1,0.0,0.0,0.1,1.485807,1.321207e-02,0.000000
6,0.995252,0.885458,1.068976,1.069052,3.800000e+01,1.000000,0.999704,0.973402,0.000214,0.253199,...,0.4,0.000000,0.0,0.1,0.0,0.0,0.1,1.754114,1.049208e-02,0.000000
7,0.995315,0.945988,1.036110,1.036276,9.864706e+01,1.000000,1.000000,0.979783,0.000257,0.217672,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,1.124316,0.000000e+00,0.000000
8,0.995115,0.904988,0.216469,0.216553,2.062500e+01,1.000000,0.970644,0.970455,0.000393,0.048890,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,0.219366,0.000000e+00,0.000000
9,0.773095,0.495313,0.196354,0.203544,8.750000e+01,1.000000,1.000000,0.962367,0.037633,0.043022,...,0.1,0.000000,0.0,0.1,0.0,0.0,0.1,0.211818,0.000000e+00,0.000000


In [18]:
scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df_featureMat_cp)

In [19]:
scaled_df = pd.DataFrame(scaled_df, columns=df_featureMat_cp.columns)

In [20]:
scaled_df

,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,issue_slot_utilization,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,0.091823,0.026259,0.054432,0.011355,1.361096e-06,1.000000,3.955402e-01,3.940969e-01,0.295849,0.013136,...,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.004542,2.644961e-04,0.000000
1,0.062160,0.001834,0.006417,0.024895,6.654245e-06,1.000000,2.409742e-01,2.345653e-01,0.107043,0.026504,...,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.009030,1.619685e-04,0.000000
2,0.100771,0.010448,0.012256,0.007948,2.586469e-06,1.000000,3.394495e-01,3.370820e-01,0.323533,0.009026,...,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.003180,1.867401e-04,0.000000
3,0.764727,0.345467,0.445156,0.471175,5.789382e-05,0.312041,5.729061e-01,5.679426e-01,0.007071,0.478160,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.365615,5.431725e-03,0.000000
4,0.995512,0.884667,0.238122,0.238972,7.134028e-07,1.000000,1.000000e+00,9.410225e-01,0.000996,0.255622,...,0.375,0.000000,0.000000,0.0,0.0,0.0,0.0,0.137218,1.938142e-02,0.000000
5,0.996504,0.873387,0.225895,0.226677,9.927079e-07,1.000000,1.000000e+00,9.765396e-01,0.000926,0.227692,...,0.500,0.000000,0.000000,0.0,0.0,0.0,0.0,0.140157,2.444248e-02,0.000000
6,0.995539,0.887531,0.259424,0.260288,8.065752e-07,1.000000,9.996949e-01,9.725435e-01,0.000943,0.279720,...,0.375,0.000000,0.000000,0.0,0.0,0.0,0.0,0.165747,1.941046e-02,0.000000
7,0.995604,0.949293,0.251277,0.252161,2.687976e-06,1.000000,1.000000e+00,9.791313e-01,0.001133,0.239831,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.105680,0.000000e+00,0.000000
8,0.995398,0.907459,0.048100,0.048904,2.675658e-07,1.000000,9.696970e-01,9.695015e-01,0.001732,0.050326,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.019370,0.000000e+00,0.000000
9,0.767476,0.489449,0.043113,0.045679,2.342170e-06,1.000000,1.000000e+00,9.611530e-01,0.165892,0.043736,...,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.018650,0.000000e+00,0.000000


### save the scaler for future use

In [21]:
metrics_scale_dd = {}
index = 0
for col_name in df_featureMat_cp.columns:
    #print col_name
    metrics_scale_dd[col_name] = [scaler.data_min_[index], scaler.data_max_[index]]
    index += 1

#print metrics_scale_dd

with codecs.open('metrics_scaler.json', 'w', encoding='utf-8') as outfile:
    json.dump(metrics_scale_dd, outfile, ensure_ascii=False)

In [22]:
metrics_scale_dd

{'achieved_occupancy': [0.015624000000000001, 0.99568400000000001],
 'atomic_transactions': [0.0, 84557.0],
 'atomic_transactions_per_request': [0.0, 10.347102],
 'branch_efficiency': [0.7297148, 1.0],
 'cf_executed': [1.0, 201988952.0],
 'cf_fu_utilization': [0.10000000000000001, 0.30000000000000004],
 'cf_issued': [1.0, 201988952.0],
 'double_precision_fu_utilization': [0.0, 1.0],
 'dram_read_throughput': [0.0, 83.787367000000003],
 'dram_read_transactions': [0.0, 1111573379.0],
 'dram_utilization': [0.0, 0.90000000000000002],
 'dram_write_throughput': [0.0, 86.053274999999999],
 'dram_write_transactions': [0.0, 9883456.0],
 'ecc_throughput': [0.0, 0.0],
 'ecc_transactions': [0.0, 0.0],
 'eligible_warps_per_cycle': [0.016272999999999999, 10.501155000000001],
 'flop_count_dp': [0.0, 11415296000.0],
 'flop_count_dp_add': [0.0, 1536768000.0],
 'flop_count_dp_fma': [0.0, 4390400000.0],
 'flop_count_dp_mul': [0.0, 1097728000.0],
 'flop_count_sp': [0.0, 51543801856.0],
 'flop_count_sp_add'

In [23]:
metrics_all = []
for metric, _ in metrics_scale_dd.iteritems():
    metrics_all.append(metric)

### save converted metrics for each application

In [25]:
metrics_folder='home-gpu0'
app_metrics_max_dd = {}
for i,f in enumerate(appTrace):
    sep_str = f.split('_')
    appName = sep_str[-1][:-4]
    
    csvfile  = metrics_folder + '/' + f
    df_app = read_trace(csvfile)
    
    #
    # find max() for current app metrics
    #
    appMetricMax_dd = {}
    for metric in metrics_all:
        df_metric = df_app.loc[df_app['Metric Name'] == metric]['Avg']
        m_list = [adjust_metric(metric, mVal) for _, mVal in df_metric.iteritems()]
        #print m_list
        appMetricMax_dd[metric] = max(m_list)  # if an applicaiton has multiple kernels, apply max() to find the val
        #break
    
    #print appMetricMax_dd
    #break
    
    app_metrics_max_dd[appName] = appMetricMax_dd # add max metrics for current app to the dd


In [26]:
# application number
appNum = len(app_metrics_max_dd)
print("total apps : {}".format(appNum))

total apps : 76


In [27]:
featMatCols = ['AppName']         
featMatCols.extend(metrics_all)

In [28]:
featMatCols

['AppName',
 'inst_control',
 'stall_constant_memory_dependency',
 'gst_efficiency',
 'inst_fp_64',
 'local_store_transactions',
 'stall_not_selected',
 'global_hit_rate',
 'stall_exec_dependency',
 'shared_store_transactions',
 'branch_efficiency',
 'warp_nonpred_execution_efficiency',
 'inst_bit_convert',
 'gst_throughput',
 'sysmem_read_throughput',
 'issue_slot_utilization',
 'shared_efficiency',
 'shared_load_transactions',
 'flop_count_dp_fma',
 'shared_store_throughput',
 'inst_executed',
 'achieved_occupancy',
 'stall_inst_fetch',
 'dram_read_transactions',
 'dram_write_transactions',
 'stall_sync',
 'gld_transactions_per_request',
 'single_precision_fu_utilization',
 'gst_requested_throughput',
 'sysmem_write_utilization',
 'flop_count_dp_mul',
 'l2_tex_read_throughput',
 'inst_per_warp',
 'inst_integer',
 'inst_issued',
 'ecc_transactions',
 'cf_fu_utilization',
 'l2_tex_read_hit_rate',
 'local_store_transactions_per_request',
 'special_fu_utilization',
 'local_load_transacti

### generate dataframe to store the metrics of each app

In [30]:
df_app = pd.DataFrame(index=np.arange(0, appNum), columns=featMatCols) #  appNum  x featureDim

#
# export data to data frame, so that we can export to csv file easily
#
rowId = 0
for appName, metrics_dd in app_metrics_max_dd.iteritems():    # read all the metrics from 13 test apps
    #print appName
    #print metrics_dd
    
    df_app.loc[rowId, 'AppName'] = appName # fill in kernel name
    # add more metrics according to the column order
    for eachMetric in metrics_all:
        try:
            df_app.loc[rowId, eachMetric] = metrics_dd[eachMetric]
        except Exception as e:
            print e.message, e.args
            print('ERROR!! App = {}'.format(appName))
            sys.exit(0)

    rowId += 1

In [31]:
df_app

,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,shared_store_transactions,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,simpleCUFFTcallback,704,0.755844,1,0,0,0.00077922,0.428571,0.138571,30,...,0.1,0.285149,0.219345,0,0.729167,0.123655,1,2,0,736
1,boxFilterNPP,337856,0.0166541,1,0,0,0.0974316,0.237288,0.127693,0,...,0.2,5.19212,5.1884,0,0.352778,0.770417,0.854492,0,0,0
2,fdtd2d,4.19635e+06,0.00050425,1,0,0,0.0135107,0.50002,0.0567325,0,...,0.3,35.5951,35.5942,0,0.999964,0.996192,1,0,0,4.1943e+06
3,scan,1.02236e+07,0.0351235,1,0,0,0.00425406,0,0.109539,482364,...,0.2,43.5654,43.5645,0,1,0.995315,1,1.5,0,0
4,binomialOptions,5.5391e+08,1.311e-05,0.125,0,0,0.100126,0,0.105748,8.38963e+06,...,0.1,0.0038147,0.0038147,0,0,0.993371,1,1,0,2.17054e+09
5,2dconv,1.67772e+07,9.844e-05,0.999512,0,0,0.00675098,0.500279,0.272958,0,...,0.5,21.9019,21.9019,0,0.857682,0.999119,0,0,0,1.34087e+08
6,mriq,1.51028e+08,0.422247,1,0,0,0.088226,0.5,0.203403,0,...,0.2,5.29271,5.21075,0,1,0.954826,1,0,0,6.69671e+08
7,gaussian,528,0.56459,0.30497,0,0,0.00092063,0.396807,0.153006,0,...,0.1,0.598907,0.528336,0,0.250195,0.209834,1,0,0,98
8,stereoDisparity,1.7495e+07,0.00017133,1,0,0,0.0157197,0,0.37452,1.64016e+06,...,0.3,0.71907,0.71907,0,0,0.99674,0,1,0,0
9,bh,1.35953e+09,0.860181,1,0,0,0.0174577,0.75,0.652332,3.44515e+06,...,0.3,119.907,119.883,50699,1,0.970295,1,1,3.7e-07,1.4443e+09


### apply feature scaling

In [38]:
# the scaling factors are stored in metrics_scale_dd
df_app_scale = df_app.copy()

for metric in metrics_all:
    [x_min, x_max] = metrics_scale_dd[metric]
    
    if x_max == x_min:
        print("[Warning] x_max == x_min ({})".format(metric))
        #x_range = 1e-6    # up-floor, avoid float division by zero
        df_app_scale[metric] = 1.0
    else:
        x_range = x_max - x_min
        df_app_scale[metric] = df_app_scale[metric].apply(lambda x : (x - x_min) / x_range)

[Warning] x_max == x_min (sysmem_read_throughput)
[Warning] x_max == x_min (sysmem_write_utilization)
[Warning] x_max == x_min (ecc_transactions)
[Warning] x_max == x_min (sysmem_read_transactions)
[Warning] x_max == x_min (sysmem_utilization)
[Warning] x_max == x_min (sysmem_read_utilization)
[Warning] x_max == x_min (ecc_throughput)
[Warning] x_max == x_min (sysmem_write_transactions)


In [42]:
# df_app_scale['sysmem_write_utilization']

In [39]:
df_app_scale

,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,shared_store_transactions,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,simpleCUFFTcallback,3.196241e-07,0.878704,1.000000,0.000000,0.000000,0.001979,0.490469,0.138573,0.000004,...,0.1,0.001893,0.001456,0.000000,0.729167,0.100771,1.000000e+00,0.345786,0.000000e+00,8.568168e-08
1,boxFilterNPP,1.536082e-04,0.019361,1.000000,0.000000,0.000000,0.247469,0.271559,0.126868,0.000000,...,0.2,0.034471,0.034446,0.000000,0.352778,0.764727,8.544922e-01,0.000000,0.000000e+00,0.000000e+00
2,fdtd2d,1.907902e-03,0.000586,1.000000,0.000000,0.000000,0.034316,0.572237,0.050516,0.000000,...,0.3,0.236320,0.236314,0.000000,0.999964,0.996504,1.000000e+00,0.000000,0.000000e+00,4.882812e-04
3,scan,4.648241e-03,0.040833,1.000000,0.000000,0.000000,0.010805,0.000000,0.107335,0.057495,...,0.2,0.289235,0.289229,0.000000,1.000000,0.995604,1.000000e+00,0.259340,0.000000e+00,0.000000e+00
4,binomialOptions,2.518393e-01,0.000015,0.125000,0.000000,0.000000,0.254313,0.000000,0.103256,1.000000,...,0.1,0.000025,0.000025,0.000000,0.000000,0.993608,1.000000e+00,0.172893,0.000000e+00,2.526838e-01
5,2dconv,7.627883e-03,0.000114,0.999512,0.000000,0.000000,0.017147,0.572533,0.283170,0.000000,...,0.5,0.145409,0.145409,0.000000,0.857682,0.999508,0.000000e+00,0.000000,0.000000e+00,1.560974e-02
6,mriq,6.866585e-02,0.490882,1.000000,0.000000,0.000000,0.224087,0.572213,0.208330,0.000000,...,0.2,0.035139,0.034595,0.000000,1.000000,0.954038,1.000000e+00,0.000000,0.000000e+00,7.795997e-02
7,gaussian,2.396044e-07,0.656362,0.304970,0.000000,0.000000,0.002338,0.454116,0.154104,0.000000,...,0.1,0.003976,0.003508,0.000000,0.250195,0.189240,1.000000e+00,0.000000,0.000000e+00,1.140870e-08
8,stereoDisparity,7.954247e-03,0.000199,1.000000,0.000000,0.000000,0.039927,0.000000,0.392448,0.195498,...,0.3,0.004774,0.004774,0.000000,0.000000,0.997067,0.000000e+00,0.172893,0.000000e+00,0.000000e+00
9,bh,6.181187e-01,1.000000,1.000000,0.000000,0.000000,0.044341,0.858320,0.691366,0.410644,...,0.3,0.796074,0.795913,0.599584,1.000000,0.969919,1.000000e+00,0.172893,3.834638e-07,1.681381e-01


In [43]:
df_app_scale.to_csv('app_proflingmetrics_scaled.csv', encoding='utf-8')